In [13]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [15]:
run = wandb.init(
    project='Fine-tune Llama 3 on CC Dataset', 
    job_type="training", 
    anonymous="allow"
)

In [16]:
base_model = "meta-llama/Llama-3.2-3B-Instruct"
new_model = "llama-3.2-3b-CC"


In [18]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [19]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)
# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.31s/it]


In [26]:
df = pd.read_csv('augmented.csv')
df

,Gender,Age,Debt,Married,BankCustomer,Industry,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,Approved,Reason
0,1,30.83,0.000,1,1,Industrials,White,1.25,1,1,1,0,ByBirth,202,0,1,"This application was approved due to Income, Y..."
1,0,58.67,4.460,1,1,Materials,Black,3.04,1,1,6,0,ByBirth,43,560,1,"This application was approved due to Income, Y..."
2,0,24.50,0.500,1,1,Materials,Black,1.50,1,0,0,0,ByBirth,280,824,1,"This application was approved due to Income, Y..."
3,1,27.83,1.540,1,1,Industrials,White,3.75,1,1,5,1,ByBirth,100,3,1,This application was approved due to YearsEmpl...
4,1,20.17,5.625,1,1,Industrials,White,1.71,1,0,0,0,ByOtherMeans,120,0,1,"This application was approved due to Income, Y..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,1,21.08,10.085,0,0,Education,Black,1.25,0,0,0,0,ByBirth,260,0,0,"This application was denied due to Employed, Z..."
686,0,22.67,0.750,1,1,Energy,White,2.00,0,1,2,1,ByBirth,200,394,0,"This application was denied due to Income, Zip..."
687,0,25.25,13.500,0,0,Healthcare,Latino,2.00,0,1,1,1,ByBirth,200,1,0,"This application was denied due to Income, Zip..."
688,1,17.92,0.205,1,1,ConsumerStaples,White,0.04,0,0,0,0,ByBirth,280,750,0,This application was denied due to YearsEmploy...


In [27]:
import pandas as pd
import random

# Function to randomly drop certain labels with a specified probability
def preprocess_data_generalized(row, drop_prob=0.2):
    # Randomly generate different text for input
    gender_text = f"Gender: {'Male' if row['Gender'] == 1 else 'Female'}" if random.random() > drop_prob else ""
    age_text = f"Age: {row['Age']}" if random.random() > drop_prob else ""
    debt_text = f"Debt: {row['Debt']}" if random.random() > drop_prob else ""
    married_text = f"Married: {'Yes' if row['Married'] == 1 else 'No'}" if random.random() > drop_prob else ""
    bank_customer_text = f"BankCustomer: {'Yes' if row['BankCustomer'] == 1 else 'No'}" if random.random() > drop_prob else ""
    industry_text = f"Industry: {row['Industry']}" if random.random() > drop_prob else ""
    ethnicity_text = f"Ethnicity: {row['Ethnicity']}" if random.random() > drop_prob else ""
    years_employed_text = f"YearsEmployed: {row['YearsEmployed']}" if random.random() > drop_prob else ""
    prior_default_text = f"PriorDefault: {'Yes' if row['PriorDefault'] == 1 else 'No'}" if random.random() > drop_prob else ""
    employed_text = f"Employed: {'Yes' if row['Employed'] == 1 else 'No'}" if random.random() > drop_prob else ""
    credit_score_text = f"CreditScore: {row['CreditScore']}" if random.random() > drop_prob else ""
    drivers_license_text = f"DriversLicense: {'Yes' if row['DriversLicense'] == 1 else 'No'}" if random.random() > drop_prob else ""
    citizen_text = f"Citizen: {row['Citizen']}" if random.random() > drop_prob else ""
    zip_code_text = f"ZipCode: {row['ZipCode']}" if random.random() > drop_prob else ""
    income_text = f"Income: {row['Income']}" if random.random() > drop_prob else ""

    # Combine input text with random omissions
    input_text = ", ".join(filter(None, [
        gender_text, age_text, debt_text, married_text, bank_customer_text, 
        industry_text, ethnicity_text, years_employed_text, prior_default_text, 
        employed_text, credit_score_text, drivers_license_text, citizen_text, 
        zip_code_text, income_text
    ]))
    
    # Simplified output format (Approved status and reason)
    output_text = f"{'Yes' if row['Approved'] == 1 else 'No'}, {row['Reason']}"
    
    return {"text": input_text, "label": output_text}

# Apply the generalized preprocessing to the dataframe
df_processed = df.apply(preprocess_data_generalized, axis=1)
df_final = pd.DataFrame(df_processed.tolist())

# Save the processed data to a new CSV file
df_final.to_csv('generalized_transformed_text_data.csv', index=False)

# Display the first few rows of the processed data
print(df_final.head())


,text,label
0,"Age: 30.83, CreditScore: 1, Income: 0, YearsEm...","Yes, This application was approved due to Inco..."
1,"Age: 58.67, CreditScore: 6, Income: 560, Years...","Yes, This application was approved due to Inco..."
2,"Age: 24.5, CreditScore: 0, Income: 824, YearsE...","Yes, This application was approved due to Inco..."
3,"Age: 27.83, CreditScore: 5, Income: 3, YearsEm...","Yes, This application was approved due to Year..."
4,"Age: 20.17, CreditScore: 0, Income: 0, YearsEm...","Yes, This application was approved due to Inco..."
...,...,...
685,"Age: 21.08, CreditScore: 0, Income: 0, YearsEm...","No, This application was denied due to Employe..."
686,"Age: 22.67, CreditScore: 2, Income: 394, Years...","No, This application was denied due to Income,..."
687,"Age: 25.25, CreditScore: 1, Income: 1, YearsEm...","No, This application was denied due to Income,..."
688,"Age: 17.92, CreditScore: 0, Income: 750, Years...","No, This application was denied due to YearsEm..."


In [28]:
dataset = Dataset.from_pandas(df_final)
dataset['text'][3]


'Age: 27.83, CreditScore: 5, Income: 3, YearsEmployed: 3.75, Gender: Male, Married: Yes, Industry: Industrials, Ethnicity: White, PriorDefault: Yes, Employed: Yes'

In [29]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)

In [30]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)
model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, peft_config)

In [31]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

In [34]:
from datasets import Dataset

# Assuming dataset is a Dataset object with columns 'text' and 'label'
train_test_split = dataset.train_test_split(test_size=0.2)  # Split into 80% train, 20% test
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",  # The column containing the input text
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)


c:\USERS\ALWAY\ANACONDA3\ENVS\CC-CHATBOT\Lib\site-packages\trl\trainer\sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
c:\USERS\ALWAY\ANACONDA3\ENVS\CC-CHATBOT\Lib\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Map: 100%|██████████| 138/138 [00:00<00:00, 6819.77 examples/s]
c:\USERS\ALWAY\ANACONDA3\ENVS\CC-CHATBOT\Lib\site-packages\wandb\sdk\wandb_run.py:2386: UserWarning: Run (xglhepcz) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
  0%|          | 0/241 [07:16<?, ?it/s]


In [ ]:
trainer.train()

  0%|          | 1/276 [00:04<18:29,  4.04s/it]

{'loss': 10.2969, 'grad_norm': 26.70298194885254, 'learning_rate': 2e-05, 'epoch': 0.0}


  1%|          | 2/276 [00:05<12:16,  2.69s/it]

{'loss': 10.3047, 'grad_norm': 26.68596839904785, 'learning_rate': 4e-05, 'epoch': 0.01}


  1%|          | 3/276 [00:07<09:24,  2.07s/it]

...

 70%|██████▉   | 193/276 [06:34<02:51,  2.06s/it]

{'loss': 0.4197, 'grad_norm': 1.2533046007156372, 'learning_rate': 6.240601503759398e-05, 'epoch': 0.7}


 70%|███████   | 194/276 [06:36<02:39,  1.94s/it]

{'loss': 0.5083, 'grad_norm': 1.1142679452896118, 'learning_rate': 6.165413533834587e-05, 'epoch': 0.7}


 71%|███████   | 195/276 [06:38<02:36,  1.93s/it]

In [ ]:
wandb.finish()

In [ ]:
# Instruction tailored to credit card approval context
instruction = """You are a highly knowledgeable financial advisor specializing in credit card approvals. 
    Be informative, polite, and provide clear responses to any queries regarding credit approval decisions.
    """

# Example message (user asking about credit card approval)
messages = [
    {"role": "system", "content": instruction},
    {"role": "user", "content": "Can I know why my credit card application was rejected? My age is 30, income is $40,000, and credit score is 580."}
]

# Generate the prompt using the chat template (assuming tokenizer.apply_chat_template is a custom method for your setup)
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

# Generate model outputs (adjusting parameters if necessary)
outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)

# Decode the model's response
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the assistant's response (assuming the response begins after the 'assistant' token)
print(text.split("assistant")[1])

# Save the fine-tuned model and tokenizer for future use
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)
